<a href="https://colab.research.google.com/github/TioHalu/Forest_Fire_Prediction_Colab/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas matplotlib numpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/tugas/dataset/weatherHistory.csv')
df.head()

In [ ]:
df['Humidity'].plot()

In [ ]:
pip install statsmodels

In [ ]:
#check stasioner data or not
from statsmodels.tsa.stattools import adfuller

In [ ]:
test_result=adfuller(df['Humidity'])

In [ ]:
def adfuller_test(Humidity):
    result=adfuller(Humidity)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )

    if result[1] <= 0.05:
      print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary")
    else:
      print("weak evidence against null hypothesis,indicating it is non-stationary ")

adfuller_test(df['Humidity'])

In [ ]:
#Let’s try to see the first difference and seasonal difference
df['Temperature (C)'] = df['Humidity'] - df['Humidity'].shift(1)
df['Temperature (C)']=df['Humidity']-df['Humidity'].shift(12)
df.head()

In [ ]:
# Again testing if data is stationary
adfuller_test(df['Temperature (C)'].dropna())

In [ ]:
df['Temperature (C)'].plot()

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['Humidity'])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Temperature (C)'].dropna(),lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Temperature (C)'].dropna(),lags=40,ax=ax2)

In [ ]:
# For non-seasonal data
#p=1, d=1, q=0 or 1

from statsmodels.tsa.arima.model import ARIMA
model=ARIMA(df['Humidity'],order=(1,1,1))
model_fit=model.fit()
model_fit.summary()

In [ ]:
df['forecast']=model_fit.predict(start=90000,end=100300,dynamic=True)
df[['Humidity','forecast']].plot(figsize=(12,8))

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(df['Humidity'],order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()
df['forecast']=results.predict(start=90000,end=100300,dynamic=True)
df[['Humidity','forecast']].plot(figsize=(12,8))

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates=[df.index[-1] + DateOffset(day=x)for x in range(0,24)]
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)

future_datest_df.tail()

future_df=pd.concat([df,future_datest_df])

future_df['forecast'] = results.predict(start = 100301, end = 100500, dynamic= True)
future_df[['Humidity', 'forecast']].plot(figsize=(12, 8))